In [1]:
using Luxor, LightXML

In [2]:
pointradius = 6
linewidth = 2;

In [3]:
function textlabel!(parent, txt::AbstractString, pos::Point; xoffset::Number=0, yoffset::Number=0)
    x = pos.x + xoffset
    y = pos.y + yoffset
    elem = new_child(parent, "text")
    add_text(elem, txt)
    set_attributes(elem, Dict("class"=>"math", "x"=>string(x, "pt"), "y"=>string(y, "pt")))
    return nothing
end;

In [4]:
function parabolicorbit()
    drawing_height = 500
    drawing_width = 250
    Drawing(drawing_width, drawing_height, :svg)
    # box(O, Point(drawing_width, drawing_height), :stroke)
    bbox = BoundingBox(;centered=false)

    origin()
    @layer begin
        setdash("dash")
        rule(O)
    end

    semilatus = 5/16*drawing_height
    @layer begin
        m1 = Point(0, 0.0)
        circle(m1, pointradius, :fill)
    end
    
    function parabola_y(y::Number, p::Number)
        return p/2 .- y.^2/(2p)
    end
    function parabola_x(x::Number, p::Number)
        return sqrt(p.^2 - 2p*x)
    end
    
    end_x = m1.x - 100
    upper_y = parabola_x(end_x, semilatus)
    lower_y = -parabola_x(end_x, semilatus)
    para_y = range(lower_y, upper_y, length=100)
    para_x = parabola_y.(para_y, semilatus)
    @layer begin
        setcolor(141/255, 160/255, 203/255, 1.0)
        para_poly = poly(Point.(para_x, para_y), :stroke)
    end

    m2x = m1.x + 70
    m2 = Point(m2x, -sqrt(semilatus^2 - 2semilatus*m2x))
    circle(m2, 2/3*pointradius, :fill)

    angle = slope(m1, m2)
    arrow(m1, m2, linewidth=linewidth)
    arrow(m1, semilatus/6, 0, angle, clockwise=false, linewidth=linewidth)

    r = distance(m1, m2)
    nu = acos(semilatus/r - 1)
    phi = nu / 2
    h = (semilatus/2) * 100.0  # arbitrary units
    mu = h^2 / semilatus
    v = sqrt(2mu/r)
    v_p = h / r
    v_r = tan(phi) * v_p
    p2 = Point(m2.x + v_r*cos(nu), m2.y - v_r*sin(nu))
    arrow(m2, p2, linewidth=linewidth)
    p4 = Point(m2.x - v_p*sin(nu), m2.y - v_p*cos(nu))
    arrow(m2, p4, linewidth=linewidth)
    arrow(m2, p2 + p4 - m2, linewidth=linewidth)

    upper_p = Point(m1.x, -semilatus)
    lower_p = rotate_point_around_point(upper_p, m1, pi)
    periapsis = Point(m1.x + semilatus/2, m1.y)
    @layer begin
        setline(linewidth/2)
        line(lower_p, upper_p, :stroke)
        offset = 25

        # Orbit parameter distance lines
        line(upper_p - (offset, 0), lower_p - (offset, 0), :stroke)
        line(upper_p - (2offset/3, 0), upper_p - (4offset/3, 0), :stroke)
        line(lower_p - (2offset/3, 0), lower_p - (4offset/3, 0), :stroke)
        line(m1 - (2offset/3, 0), m1 - (4offset/3, 0), :stroke)
        
        # Periapsis distance lines
        line(m1 + (0, offset), periapsis + (0, offset), :stroke)
        line(m1 + (0, 2offset/3), m1 + (0, 4offset/3), :stroke)
        line(periapsis + (0, 2offset/3), periapsis + (0, 4offset/3), :stroke)

#         offset *= 2
#         # Apsis distance lines
#         line(Point(-semimajor, offset), Point(semimajor, offset), :stroke)
#         line(Point(semimajor, 14offset/15), Point(semimajor, 16*offset/15), :stroke)
#         line(Point(-semimajor, 14offset/15), Point(-semimajor, 16*offset/15), :stroke)
#         line(Point(m1.x, 14offset/15), Point(m1.x, 16*offset/15), :stroke)

#         offset /= 2
    end
    @layer begin
        setcolor(252/255, 141/255, 98/255, 1.0)
        circle(upper_p, pointradius/2, :fill)
        circle(lower_p, pointradius/2, :fill)
        circle(periapsis, pointradius/2, :fill)
    end
    finish()
    svgout = svgstring()
    xdoc = parse_string(svgout)
    xroot = root(xdoc)
    textlabel!(xroot, "\\boldsymbol{r}", between(m1, m2, 0.4), xoffset=90, yoffset=175)
    textlabel!(xroot, "\\boldsymbol{v}", between(m1, m2, 0.4), xoffset=100, yoffset=100)
    textlabel!(xroot, "v_r", m2, xoffset=95, yoffset=185)
    textlabel!(xroot, "v_{\\perp}", between(m1, m2, 0.4), xoffset=90, yoffset=150)
    textlabel!(xroot, "m_1", m1, xoffset=77, yoffset=195)
    textlabel!(xroot, "m_2", m2, xoffset=77, yoffset=205)
    textlabel!(xroot, "\\nu", between(m1, m2, 0.85), xoffset=60, yoffset=220)
    textlabel!(xroot, "p", between(upper_p, m1, 0.5), xoffset=60, yoffset=200)
    textlabel!(xroot, "p", between(lower_p, m1, 0.5), xoffset=60, yoffset=170)
    textlabel!(xroot, "p/2", between(m1, periapsis, 0.5), xoffset=70, yoffset=210)
    save_file(xdoc, "../raw_svg/parabolic-orbit-definitions.svg");
end

parabolicorbit();